In [1]:
import plotly.express as px
import pandas as pd

from src import settings
from src.visualization.plot import line
from src.data.make_knn_dataset import get_organ_labels

/home/menjivar/anaconda3/envs/miccai23/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


# plot organ spectra

In [2]:
spectra_file = settings.figures_dir / 'semantic_reflectance.csv'
df = pd.read_csv(spectra_file)

In [17]:
def prepare_data(df: pd.DataFrame):
    to_replace = {
        'simulated_adapted': 'synthetic adapted',
        'simulated_sampled': 'synthetic sampled',
        'small_bowel': 'small bowel'
    }
    to_rename = {
        'wavelength': 'wavelength [nm]',
        'dataset': 'data'
    }
    new_df = df.replace(to_replace, inplace=False)
    new_df.rename(to_rename, inplace=True, axis=1)
    to_rename.update(to_replace)
    return new_df, to_rename

In [30]:
organs = get_organ_labels()['organ_labels']
organs = [' '.join(o.split('_')) for o in organs]
df_prepared, mapper = prepare_data(df)
fig, plot_data = line(
    data_frame=df_prepared,
    x=mapper.get("wavelength"),
    y="reflectance",
    facet_col="organ",
    color=mapper.get("dataset"),
    facet_col_wrap=5,
    template="plotly_white",
    width=800,
    height=400,
    category_orders=dict(organ=organs),
    facet_row_spacing=0.2
)
fig.update_layout(font=dict(size=12, family="Whitney Book"))
fig.update_xaxes(title_font=dict(size=12, family="Whitney Book"))
fig.update_yaxes(title_font=dict(size=12, family="Whitney Book"))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.write_image(settings.figures_dir / 'manuscript' / 'organ_reflectance.pdf')